# Export from Pandas to Excel

In [1]:
import pandas as pd

### Create two dataframes

In [2]:
#Create two dataframes

data = [
    ["One", "Two", "Three"],
    [1,2,3],
    [4,5,6],
    [7,8,9]
    ]
headers = data.pop(0)

df1 = pd.DataFrame(data, columns=headers)

data = {"x": range(0,6)}
df2 = pd.DataFrame(data)
df2["y"] = df2["x"]**2
df2["z"] = 5*df2["x"] - df2["y"]
display(df1, df2)

,One,Two,Three
0,1,2,3
1,4,5,6
2,7,8,9


,x,y,z
0,0,0,0
1,1,1,4
2,2,4,6
3,3,9,6
4,4,16,4
5,5,25,0


### Some highlighting

In [5]:
def highlight(val):
    """Different way of mapping the colours"""
    mapping = {
        0:("red", "pink"),
        1:("yellow", "orange"),
        4:("purple", "lightblue")
    }
    colours = mapping.get(val, ("black", "white") )
    return 'color: {}; background-color: {}'.format(*colours)

styles = [
    {"selector":"th", "props":[("font-size","120%"),("color","cyan"),("background-color","dimgrey")]}
]


dfh = df2.style.applymap(highlight)\
    .set_table_styles(styles)\
    .set_properties(**{'border-color': 'black', 'border-style':'solid', 'border-width':'thin'})
display(dfh)

,x,y,z
0,0,0,0
1,1,1,4
2,2,4,6
3,3,9,6
4,4,16,4
5,5,25,0


### (Diversion) To HTML as alternative

In [19]:
# Doesn't work with stylers, only dataframes
# However, alternative approach for stylers .render() https://stackoverflow.com/questions/36897366/pandas-to-html-using-the-style-options-or-custom-css/37760101
##dfh.to_html("df.html")
df1_table = df1.to_html() # can specify filename to write directly to html file

In [18]:
from IPython.core.display import  HTML
display(HTML(df1_table))

,One,Two,Three
0,1,2,3
1,4,5,6
2,7,8,9


## Save to Excel
df1.to_excel followed by spreadsheet.save() doesn't support cumulative updates, seems  doing save clears all data from ram, so second save results in spreadsheet with only details from between the two saves.

Possibly using openpyxl is better alternative.

In [15]:
filename = "temp.xlsx"
spreadsheet = pd.ExcelWriter(filename)

df1.to_excel(spreadsheet, "Results", startrow=3, startcol=1, index=False)
spreadsheet.save()
df2.to_excel(spreadsheet, "Results", startrow=3, startcol=6, index=False)
spreadsheet.save()

/usr/lib/python2.7/dist-packages/pandas/io/formats/excel.py:309: CSSWarning: Unhandled color format: 'pink'
  CSSWarning)
/usr/lib/python2.7/dist-packages/pandas/io/formats/excel.py:309: CSSWarning: Unhandled color format: 'lightblue'
  CSSWarning)


In [ ]:
def df_to_spreadsheet(df, spreadsheet, sheet_name="Results", startrow=3,
                      startcol=1, heading="Result", colwidths=(), **kwargs):
    """Write dataframe to sparticular location within a spreadsheet.
    Optionally set a heading and column widths
    Args:
        df - dataframe (or style object) to be written
        spreadsheet - xlfilewriter spreadsheet object
        startrow - topmost row to start from
        startcol - leftmost column to start from
        heading - heading text (placed above startrow)
        colwidths - list/tuple of column widths in order for the dataframe rows
                    can be less than total of empty
                    
        (**kwargs to enable arguments from dictionary with surplus elements)
    """
    ##spreadsheet = pd.ExcelWriter(filename)
    #Way to get bold style for writing contents to cell
    bold = spreadsheet.book.add_format({'bold': 1})

    # Write results to spreadsheet (index=False stops df index values being written)
    df.to_excel(spreadsheet, sheet_name, startrow=startrow, startcol=startcol, index=False)

    #Set heading and column widths
    ws = spreadsheet.sheets[sheet_name]
    ws.hide_gridlines(2)

    #Add heading text
    ##ws.write('A1', time.strftime("Unfinished Hub Jobs %d/%m/%Y %H:%M:%S"), bold)
    ws.write(startrow-1, startcol, time.strftime(query["Heading"]+" %d/%m/%Y %H:%M:%S"), bold)

    #Adjust column widths/styling:
    for i, w in enumerate(colwidths):
        #Can specify columns by letter "C:D" or number 3,5
        ws.set_column(startcol+i, startcol+i, w)

    #Save the spreadsheet - seems can only save once. Multiple saves results in just the last changes 
    ##spreadsheet.save()

## Alternative Approach - openpyxl

In [ ]:
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

filename = "temp2.xlsx"

wb = Workbook()
ws = wb.active

for row in dataframe_to_rows(df1, index=False, header=True):
    ws.append(row)

for row in dataframe_to_rows(df2, index=False, header=True):
    ws.append(row)
    
wb.save(filename = filename)